In [41]:
import warnings
warnings.filterwarnings('ignore')
import os
from sqlalchemy import create_engine, Table
from daeshin.models.minutepricemodel import MinutePriceModel
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from config.settings import META

In [2]:
load_dotenv()

True

In [3]:
LOCAL_DAESHIN_DATABASE = os.environ.get("LOCAL_DAESHIN_DATABASE")

In [5]:
engine = create_engine(LOCAL_DAESHIN_DATABASE)

In [6]:
conn = engine.connect()

In [16]:
file_paths = Path('data').glob("*.pkl")

In [17]:
tmp = list(file_paths)

In [33]:
def rename_columns(x: str):
    if "_price" in x:
        return x.replace("_price", "")
    else:
        return x

In [38]:
ticker = tmp[0].stem
data = pd.read_pickle(tmp[0])
data.rename(columns=rename_columns, inplace=True)

In [39]:
def insert_minute_price(path):
    LOCAL_DAESHIN_DATABASE = os.environ.get("LOCAL_DAESHIN_DATABASE")
    engine = create_engine(LOCAL_DAESHIN_DATABASE)
    conn = engine.connect()
    minute_table = Table(MinutePriceModel, META, autoload_with=engine)
    price = pd.read_pickle(path)
    price.rename(columns=rename_columns, inplace=True)
    price_dict = price.to_dict('records')
    conn.execute(minute_table.insert().prefix_with("IGNORE"), price_dict)
    del conn


,date,open,high,low,close,volume,transaction_amount,ticker
0,2019-03-25 09:01:00,9600,9600,9590,9590,1394,13380000,000020
1,2019-03-25 09:02:00,9590,9590,9590,9590,954,9150000,000020
2,2019-03-25 09:04:00,9590,9590,9580,9580,52,490000,000020
3,2019-03-25 09:05:00,9580,9580,9560,9560,549,5250000,000020
4,2019-03-25 09:06:00,9560,9560,9550,9550,904,8640000,000020
...,...,...,...,...,...,...,...,...
190061,2021-10-08 15:17:00,20150,20150,19100,19400,297553,5801920000,000020
190062,2021-10-08 15:18:00,19400,19450,18800,18850,218788,4174270000,000020
190063,2021-10-08 15:19:00,18850,19400,18800,19350,135698,2605120000,000020
190064,2021-10-08 15:20:00,19350,19550,19250,19500,87431,1698100000,000020
